# Experiment 002: Extended Optimization with Fractional Translation

This experiment implements:
1. Fractional translation post-processing (very fine step sizes)
2. Focus on small N values (1-20) which contribute most to score
3. Validate for overlaps

In [1]:
import numpy as np
import pandas as pd
import os
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union
from shapely.strtree import STRtree
from scipy.spatial import ConvexHull
from scipy.optimize import minimize_scalar
import time
from numba import njit
import math

getcontext().prec = 30
scale_factor = 1

os.chdir('/home/code/experiments/002_extended_optimization')
print(f'Working directory: {os.getcwd()}')

Working directory: /home/code/experiments/002_extended_optimization


In [2]:
# Tree polygon template
@njit
def make_polygon_template():
    tw=0.15; th=0.2; bw=0.7; mw=0.4; ow=0.25
    tip=0.8; t1=0.5; t2=0.25; base=0.0; tbot=-th
    x=np.array([0,ow/2,ow/4,mw/2,mw/4,bw/2,tw/2,tw/2,-tw/2,-tw/2,-bw/2,-mw/4,-mw/2,-ow/4,-ow/2],np.float64)
    y=np.array([tip,t1,t1,t2,t2,base,base,tbot,tbot,base,base,t2,t2,t1,t1],np.float64)
    return x, y

@njit
def get_tree_polygon(cx, cy, deg, tx, ty):
    """Get tree polygon vertices at position (cx, cy) with rotation deg."""
    r = deg * math.pi / 180.0
    c = math.cos(r)
    s = math.sin(r)
    n = len(tx)
    px = np.empty(n, dtype=np.float64)
    py = np.empty(n, dtype=np.float64)
    for i in range(n):
        px[i] = c * tx[i] - s * ty[i] + cx
        py[i] = s * tx[i] + c * ty[i] + cy
    return px, py

@njit
def get_bbox(xs, ys, degs, tx, ty):
    """Get bounding box side length for a configuration."""
    n = xs.size
    V = tx.size
    mnx = 1e300; mny = 1e300; mxx = -1e300; mxy = -1e300
    for i in range(n):
        r = degs[i] * math.pi / 180.0
        c = math.cos(r)
        s = math.sin(r)
        xi = xs[i]
        yi = ys[i]
        for j in range(V):
            X = c * tx[j] - s * ty[j] + xi
            Y = s * tx[j] + c * ty[j] + yi
            if X < mnx: mnx = X
            if X > mxx: mxx = X
            if Y < mny: mny = Y
            if Y > mxy: mxy = Y
    return max(mxx - mnx, mxy - mny)

@njit
def score_group(xs, ys, degs, tx, ty):
    """Calculate score for a group."""
    n = xs.size
    side = get_bbox(xs, ys, degs, tx, ty)
    return side * side / n

tx, ty = make_polygon_template()
print('Template created')

Template created


In [3]:
# Load baseline submission
import shutil
shutil.copy('/home/nonroot/snapshots/santa-2025/21105319338/code/datasets/santa-2025-csv/santa-2025.csv', 'submission.csv')

def strip(a):
    return np.array([float(str(v).replace('s', '')) for v in a], np.float64)

def load_submission(filepath):
    df = pd.read_csv(filepath)
    df['N'] = df['id'].astype(str).str.split('_').str[0].astype(int)
    configs = {}
    for n, g in df.groupby('N'):
        xs = strip(g['x'].to_numpy())
        ys = strip(g['y'].to_numpy())
        ds = strip(g['deg'].to_numpy())
        configs[n] = {'x': xs, 'y': ys, 'deg': ds, 'df': g.copy()}
    return configs

configs = load_submission('submission.csv')

# Calculate initial score
initial_score = 0.0
for n in range(1, 201):
    if n in configs:
        c = configs[n]
        initial_score += score_group(c['x'], c['y'], c['deg'], tx, ty)

print(f'Initial score: {initial_score:.6f}')

Initial score: 70.734327


In [4]:
# Overlap detection using Shapely
def get_shapely_polygon(cx, cy, deg, tx, ty):
    """Create Shapely polygon for a tree."""
    r = deg * np.pi / 180.0
    c = np.cos(r)
    s = np.sin(r)
    px = c * tx - s * ty + cx
    py = s * tx + c * ty + cy
    return Polygon(zip(px, py))

def has_overlap(xs, ys, degs, tx, ty):
    """Check if any trees overlap."""
    n = len(xs)
    if n <= 1:
        return False
    
    polygons = [get_shapely_polygon(xs[i], ys[i], degs[i], tx, ty) for i in range(n)]
    tree_index = STRtree(polygons)
    
    for i, poly in enumerate(polygons):
        indices = tree_index.query(poly)
        for idx in indices:
            if idx == i:
                continue
            if poly.intersects(polygons[idx]) and not poly.touches(polygons[idx]):
                return True
    return False

print('Overlap detection defined')

Overlap detection defined


In [5]:
# Fractional translation optimization
@njit
def fractional_translation_step(xs, ys, degs, tx, ty, tree_idx, step, direction):
    """Try moving one tree by a small step in one direction."""
    dx = np.array([0.0, 0.0, 1.0, -1.0, 1.0, 1.0, -1.0, -1.0])
    dy = np.array([1.0, -1.0, 0.0, 0.0, 1.0, -1.0, 1.0, -1.0])
    
    new_xs = xs.copy()
    new_ys = ys.copy()
    new_xs[tree_idx] += dx[direction] * step
    new_ys[tree_idx] += dy[direction] * step
    
    return new_xs, new_ys

def fractional_translation(xs, ys, degs, tx, ty, max_iter=200):
    """Apply fractional translation optimization."""
    best_xs = xs.copy()
    best_ys = ys.copy()
    best_side = get_bbox(best_xs, best_ys, degs, tx, ty)
    
    frac_steps = [0.001, 0.0005, 0.0002, 0.0001, 0.00005, 0.00002, 0.00001]
    n = len(xs)
    
    for iteration in range(max_iter):
        improved = False
        for i in range(n):
            for step in frac_steps:
                for d in range(8):
                    new_xs, new_ys = fractional_translation_step(best_xs, best_ys, degs, tx, ty, i, step, d)
                    
                    # Check for overlap
                    if has_overlap(new_xs, new_ys, degs, tx, ty):
                        continue
                    
                    new_side = get_bbox(new_xs, new_ys, degs, tx, ty)
                    if new_side < best_side - 1e-12:
                        best_side = new_side
                        best_xs = new_xs
                        best_ys = new_ys
                        improved = True
        
        if not improved:
            break
    
    return best_xs, best_ys, best_side

print('Fractional translation defined')

Fractional translation defined


In [6]:
# Apply fractional translation to small N values (1-20)
print('Applying fractional translation to small N values...')
start_time = time.time()

improved_configs = {}
for n in range(1, 21):
    if n not in configs:
        continue
    
    c = configs[n]
    old_score = score_group(c['x'], c['y'], c['deg'], tx, ty)
    
    new_xs, new_ys, new_side = fractional_translation(c['x'], c['y'], c['deg'], tx, ty, max_iter=100)
    new_score = new_side * new_side / n
    
    if new_score < old_score - 1e-12:
        print(f'N={n}: {old_score:.8f} -> {new_score:.8f} (improved by {old_score - new_score:.8f})')
        improved_configs[n] = {'x': new_xs, 'y': new_ys, 'deg': c['deg']}
    else:
        print(f'N={n}: {old_score:.8f} (no improvement)')

print(f'\nTime: {time.time() - start_time:.1f}s')

Applying fractional translation to small N values...
N=1: 0.66125000 (no improvement)
N=2: 0.45077918 (no improvement)


N=3: 0.43474514 (no improvement)


N=4: 0.41654485 (no improvement)
N=5: 0.41684959 (no improvement)


N=6: 0.39961029 (no improvement)


N=7: 0.39989655 (no improvement)


...

N=16: 0.37412799 (no improvement)


N=17: 0.37004030 (no improvement)


N=18: 0.36877123 (no improvement)


N=19: 0.36861533 (no improvement)


N=20: 0.37605686 (no improvement)

Time: 8.4s


In [7]:
# Update configs with improvements
for n, new_config in improved_configs.items():
    configs[n]['x'] = new_config['x']
    configs[n]['y'] = new_config['y']

# Calculate new total score
new_score = 0.0
for n in range(1, 201):
    if n in configs:
        c = configs[n]
        new_score += score_group(c['x'], c['y'], c['deg'], tx, ty)

print(f'Initial score: {initial_score:.6f}')
print(f'New score: {new_score:.6f}')
print(f'Improvement: {initial_score - new_score:.6f}')

Initial score: 70.734327
New score: 70.734327
Improvement: 0.000000


In [8]:
# Save updated submission
def save_submission(configs, filepath):
    rows = []
    for n in range(1, 201):
        if n in configs:
            c = configs[n]
            for i in range(len(c['x'])):
                rows.append({
                    'id': f'{n:03d}_{i}',
                    'x': f's{c["x"][i]}',
                    'y': f's{c["y"][i]}',
                    'deg': f's{c["deg"][i]}'
                })
    df = pd.DataFrame(rows)
    df.to_csv(filepath, index=False)
    print(f'Saved to {filepath}')

save_submission(configs, 'submission.csv')

Saved to submission.csv


In [9]:
# Validate for overlaps
print('Validating for overlaps...')
overlap_n = []
for n in range(1, 201):
    if n in configs:
        c = configs[n]
        if has_overlap(c['x'], c['y'], c['deg'], tx, ty):
            overlap_n.append(n)

if overlap_n:
    print(f'Overlaps found in N: {overlap_n}')
else:
    print('No overlaps detected')

Validating for overlaps...


No overlaps detected


In [10]:
# Copy to submission folder
import shutil
shutil.copy('submission.csv', '/home/submission/submission.csv')
print('Submission saved to /home/submission/submission.csv')

print(f'\n=== FINAL RESULTS ===')
print(f'Initial score: {initial_score:.6f}')
print(f'Final score: {new_score:.6f}')
print(f'Improvement: {initial_score - new_score:.6f}')

Submission saved to /home/submission/submission.csv

=== FINAL RESULTS ===
Initial score: 70.734327
Final score: 70.734327
Improvement: 0.000000


In [11]:
# Search for all submission files in snapshots and find best per-N configurations\nimport glob\n\nprint('Searching for submission files in snapshots...')\nall_files = glob.glob('/home/nonroot/snapshots/**/*.csv', recursive=True)\nprint(f'Found {len(all_files)} CSV files')\n\n# Filter for submission-like files\nsubmission_files = [f for f in all_files if 'submission' in f.lower() or 'santa' in f.lower()]\nprint(f'Found {len(submission_files)} potential submission files')

In [12]:
# Search for all submission files in snapshots and find best per-N configurations
import glob

print('Searching for submission files in snapshots...')
all_files = glob.glob('/home/nonroot/snapshots/**/*.csv', recursive=True)
print(f'Found {len(all_files)} CSV files')

# Filter for submission-like files
submission_files = [f for f in all_files if 'submission' in f.lower() or 'santa' in f.lower()]
print(f'Found {len(submission_files)} potential submission files')

Searching for submission files in snapshots...
Found 235 CSV files
Found 235 potential submission files


In [14]:
# Find best per-N configurations from all files
from tqdm import tqdm

best = {n: {'score': 1e300, 'data': None, 'src': None} for n in range(1, 201)}

for fp in tqdm(submission_files, desc='Scanning'):
    try:
        df = pd.read_csv(fp)
    except Exception:
        continue
    
    if not {'id', 'x', 'y', 'deg'}.issubset(df.columns):
        continue
    
    # Check if values have 's' prefix
    sample_x = str(df['x'].iloc[0])
    if not sample_x.startswith('s'):
        continue
    
    df = df.copy()
    df['N'] = df['id'].astype(str).str.split('_').str[0].astype(int)
    
    for n, g in df.groupby('N'):
        if n < 1 or n > 200:
            continue
        
        xs = strip(g['x'].to_numpy())
        ys = strip(g['y'].to_numpy())
        ds = strip(g['deg'].to_numpy())
        
        # Check for overlaps
        if has_overlap(xs, ys, ds, tx, ty):
            continue
        
        sc = score_group(xs, ys, ds, tx, ty)
        if sc < best[n]['score']:
            best[n]['score'] = float(sc)
            best[n]['data'] = g.drop(columns=['N']).copy()
            best[n]['src'] = fp

print('\nBest scores found:')
for n in range(1, 21):
    if best[n]['data'] is not None:
        print(f'N={n}: {best[n]["score"]:.8f} from {best[n]["src"].split("/")[-1]}')

Scanning:   0%|          | 0/235 [00:00<?, ?it/s]

Scanning:   0%|          | 1/235 [00:01<06:30,  1.67s/it]

Scanning:   1%|          | 2/235 [00:03<06:14,  1.61s/it]

Scanning:   1%|▏         | 3/235 [00:04<06:09,  1.59s/it]

Scanning:   2%|▏         | 4/235 [00:06<05:47,  1.50s/it]

...

Scanning:  99%|█████████▉| 233/235 [04:59<00:02,  1.15s/it]

Scanning: 100%|█████████▉| 234/235 [05:00<00:01,  1.01s/it]

Scanning: 100%|██████████| 235/235 [05:02<00:00,  1.16s/it]

Scanning: 100%|██████████| 235/235 [05:02<00:00,  1.29s/it]


Best scores found:
N=1: 0.66125000 from submission.csv
N=2: 0.45077918 from ensemble_submission.csv
N=3: 0.43474514 from submission.csv
N=4: 0.41654485 from submission.csv
N=5: 0.41684959 from submission.csv
N=6: 0.39961029 from submission.csv
N=7: 0.39989655 from submission.csv
N=8: 0.38540725 from submission.csv
N=9: 0.38741503 from submission.csv
N=10: 0.37663004 from submission.csv
N=11: 0.37573607 from submission.csv
N=12: 0.37272399 from submission.csv
N=13: 0.37232314 from submission.csv
N=14: 0.37111261 from submission.csv
N=15: 0.37920286 from submission.csv
N=16: 0.37412799 from submission.csv
N=17: 0.37004030 from submission.csv
N=18: 0.36877123 from submission.csv
N=19: 0.36861533 from submission.csv
N=20: 0.37605686 from submission.csv
